In [1]:
!pip install -q timesketch_api_client

In [8]:
from timesketch_api_client import search
from picatrix.lib import state as state_lib

import altair as alt
import numpy as np
import pandas as pd
from timesketch_api_client import config
from timesketch_import_client import helper
from timesketch_import_client import importer
from timesketch_api_client import search

# Create the sketch and import some data

In [ ]:
ts_client = config.get_client()

In [ ]:
sketch = ts_client.create_sketch(name="test")

In [3]:
# Download a file to import
import requests
import io

url = "https://raw.githubusercontent.com/google/timesketch/master/test_tools/test_events/sigma_events.csv" 
download = requests.get(url).content

In [10]:
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df

,message,timestamp,datetime,timestamp_desc,extra_field_1,command,data_type,display_name,filename,packages,parser
0,A message,123456789,2015-07-24T19:01:01+00:00,Write time,foo,NaN,NaN,NaN,NaN,NaN,NaN
1,Another message,123456790,2015-07-24T19:01:02+00:00,Write time,bar,NaN,NaN,NaN,NaN,NaN,NaN
2,Yet more messages,123456791,2015-07-24T19:01:03+00:00,Write time,baz,NaN,NaN,NaN,NaN,NaN,NaN
3,Install: zmap:amd64 (1.1.0-1) [Commandline: ap...,123456791,2015-07-24T19:01:03+00:00,foo,NaN,Commandline: apt-get install zmap,apt:history:line,GZIP:/var/log/apt/history.log.1.gz,/var/log/apt/history.log.1.gz,Install: zmap:amd64 (1.1.0-1),apt_history


In [ ]:
import_helper = helper.ImportHelper() 

with importer.ImportStreamer() as streamer:
  streamer.set_sketch(sketch)
  streamer.set_config_helper(import_helper) 

  streamer.set_timeline_name('sigma_events.csv')
  streamer.add_data_frame(df)


# Multiple Rules


In [ ]:
rules = ts_client.list_sigma_rules()

In [ ]:
for rule in rules:
    rule.id

In [ ]:
rule1 = rules[0]

In [ ]:
rule1.id

In [ ]:
rule1.es_query

# Single rule

In [ ]:
# Single Rule tests
single_rule = ts_client.get_sigma_rule(rule_uuid='5266a592-b793-11ea-b3de-0242ac130004')

In [ ]:
single_rule.id

In [ ]:
single_rule.rule_uuid


In [ ]:
single_rule.es_query

# Text Sigma examples

In [ ]:
MOCK_SIGMA_RULE = """
title: Suspicious Installation of Zenmap
id: 5266a592-b793-11ea-b3de-0242ac130004
description: Detects suspicious installation of Zenmap
references:
    - https://rmusser.net/docs/ATT&CK-Stuff/ATT&CK/Discovery.html
author: Alexander Jaeger
date: 2020/06/26
modified: 2021/01/01
logsource:
    product: linux
    service: shell
detection:
    keywords:
        # Generic suspicious commands
        - '*apt-get install zmap*'
    condition: keywords
falsepositives:
    - Unknown
level: high
"""

In [ ]:
rule_by_text = ts_client.get_sigma_rule_by_text(MOCK_SIGMA_RULE)

In [ ]:
rule_by_text.es_query

In [ ]:
rule_by_text.references

# Analyzer

In [ ]:
timelines = sketch.list_timelines()
timeline = None
for timeline_ in timelines:
  if timeline_.name == 'sigma_events.csv':
    timeline = timeline_
    break

In [ ]:
result = timeline.run_analyzer(analyzer_name='sigma', ignore_previous=True)


In [ ]:
sketch.get_analyzer_status()
"""This should show something like:

[{'index': <timesketch_api_client.index.SearchIndex at 0x126a09820>,
  'timeline_id': 1,
  'session_id': 1,
  'analyzer': 'sigma',
  'results': 'Applied 1 tags\n* lnx_susp_zenmap.yml: 1\nProblematic rules:',
  'status': 'DONE'}]
  """